In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# https://www.kaggle.com/rajatranjan/fork-of-mh-wipro-ml-challenge-reg?scriptVersionId=86535459

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mh-wipro-sustainable-ml-challenge/sample_submission.csv
/kaggle/input/mh-wipro-sustainable-ml-challenge/train.csv
/kaggle/input/mh-wipro-sustainable-ml-challenge/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/mh-wipro-sustainable-ml-challenge/train.csv")
test = pd.read_csv("/kaggle/input/mh-wipro-sustainable-ml-challenge/test.csv")
sample = pd.read_csv("/kaggle/input/mh-wipro-sustainable-ml-challenge/sample_submission.csv")

In [3]:
# !pip install tsfel

In [4]:
dt_cols=['Year', 'Month', 'Day', 'Hour', 'Minute']
# train['date'] = pd.to_datetime(train[dt_cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns"),format="%Y-%m-%d-%H-%M")
# test['date'] = pd.to_datetime(test[dt_cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns"),format="%Y-%m-%d-%H-%M")
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175296 entries, 0 to 175295
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Year                175296 non-null  int64  
 1   Month               175296 non-null  int64  
 2   Day                 175296 non-null  int64  
 3   Hour                175296 non-null  int64  
 4   Minute              175296 non-null  int64  
 5   Clearsky DHI        175296 non-null  int64  
 6   Clearsky DNI        175296 non-null  int64  
 7   Clearsky GHI        175296 non-null  int64  
 8   Cloud Type          175296 non-null  int64  
 9   Dew Point           175296 non-null  float64
 10  Temperature         175296 non-null  float64
 11  Pressure            175296 non-null  int64  
 12  Relative Humidity   175296 non-null  float64
 13  Solar Zenith Angle  175296 non-null  float64
 14  Precipitable Water  175296 non-null  float64
 15  Wind Direction      175296 non-nul

In [5]:
train = train[train['Cloud Type']<10].copy()
train['Cloud Type'].replace(1,0,inplace=True)

In [6]:
from tqdm import tqdm
# for col in tqdm(['Wind Speed','Wind Direction','Temperature','Dew Point','Relative Humidity','Solar Zenith Angle','Precipitable Water','Pressure']):
#     for k in tqdm(train['Year'].unique()):
#         for mth,day,low_qn,high_qn in tmp_test[['Month','Day',col+'_min',col+'_max']].values:
#             tmp_df = train.query(f'Year=={k} & Month=={mth} & Day=={day}')
#             tmp_df[col] = np.clip(tmp_df[col],low_qn,high_qn)
#             train.loc[tmp_df.index,col] = tmp_df[col]

In [7]:
train.describe()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
count,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000
mean,2013.500265,6.523552,15.727750,11.500009,15.000086,53.065729,330.067178,254.366941,2.450880,13.564377,18.266742,1003.185248,79.376249,89.729423,2.903719,188.877277,2.262875,0.740438
std,2.871914,3.448553,8.799326,6.922225,15.000043,64.201465,370.031036,327.705377,3.080409,8.196780,8.943115,5.935761,19.678230,40.085562,1.546974,99.014777,1.281905,6.575674
min,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-17.000000,-9.000000,970.000000,7.190000,8.550000,0.122000,0.000000,0.000000,0.000000
25%,2011.000000,4.000000,8.000000,5.500000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,12.000000,1000.000000,65.590000,56.450000,1.534000,97.400000,1.300000,0.000000
50%,2013.000000,7.000000,16.000000,12.000000,30.000000,0.000000,0.000000,0.000000,0.000000,15.000000,20.000000,1000.000000,84.230000,89.860000,2.866000,200.000000,2.000000,0.000000
75%,2016.000000,10.000000,23.000000,17.500000,30.000000,103.000000,736.000000,535.000000,6.000000,21.000000,25.000000,1010.000000,97.950000,123.060000,4.223000,272.000000,3.000000,0.000000
max,2018.000000,12.000000,31.000000,23.000000,30.000000,565.000000,1040.000000,1063.000000,9.000000,26.900000,40.000000,1024.000000,100.000000,171.550000,7.700000,360.000000,12.900000,100.000000


In [8]:
# def outlier_treatment(datacolumn):
#     sorted(datacolumn)
#     Q1,Q3 = np.percentile(datacolumn , [25,75])
#     IQR = Q3 — Q1
#     lower_range = Q1 — (1.5 * IQR)
#     upper_range = Q3 + (1.5 * IQR)
#     return lower_range,upper_range

# 50th Percentile
def q01(x):
    return x.quantile(0.03)

# 90th Percentile
def q99(x):
    return x.quantile(0.97)


tmp_quantiles = train.groupby(['Year','Month','Day']).agg({
     'Dew Point':[q01,q99],
       'Temperature':[q01,q99], 'Pressure':[q01,q99], 'Relative Humidity':[q01,q99], 'Solar Zenith Angle':[q01,q99],
       'Precipitable Water':[q01,q99], 'Wind Direction':[q01,q99], 'Wind Speed':[q01,q99],
    
})
tmp_quantiles.columns = ['_'.join(col).strip() for col in tmp_quantiles.columns.values]
tmp_quantiles

Dew Point_q01  Dew Point_q99  Temperature_q01  \
Year Month Day                                                  
2009 1     1           -4.000          1.590           -1.000   
           2            0.410         15.000            3.000   
           3           11.000         17.000           12.000   
           4           11.820         16.590           12.410   
           5           11.000         17.590           12.000   
...                       ...            ...              ...   
2018 12    27           6.700         16.400            8.141   
           28          17.182         21.400           17.341   
           29          15.382         18.859           15.582   
           30          15.300         20.859           15.300   
           31          15.700         20.018           15.700   

                Temperature_q99  Pressure_q01  Pressure_q99  \
Year Month Day                                                
2009 1     1              9.000        1010.0        1010.0   
           2             18.000        1000.0        1010.0   
           3             20.000        1000.0        1000.0   
           4             21.000        1000.0        1010.0   
           5             22.000        1000.0        1010.0   
...                         ...           ...           ...   
2018 12    27            16.859        1008.0        1016.0   
           28            21.959        1005.0        1008.0   
           29            18.859        1008.0        1013.0   
           30            22.700        1008.0        1011.0   
           31            23.100        1006.0        1010.0   

                Relative Humidity_q01  Relative Humidity_q99  \
Year Month Day                                                 
2009 1     1                  43.6597                92.2459   
           2                  81.4793               100.0000   
           3                  89.5569               100.0000   
           4                  75.9181               100.0000   
           5                  77.5623               100.0000   
...                               ...                    ...   
2018 12    27                 89.7891                99.2259   
           28                 95.0724               100.0000   
           29                 88.2540               100.0000   
           30                 88.7000               100.0000   
           31                 80.3479               100.0000   

                Solar Zenith Angle_q01  Solar Zenith Angle_q99  \
Year Month Day                                                   
2009 1     1                   55.2478                169.0165   
           2                   55.1601                168.9414   
           3                   55.0624                168.8604   
           4                   54.9606                168.7694   
           5                   54.8529                168.6784   
...                                ...                     ...   
2018 12    27                  55.5940                169.2871   
           28                  55.5404                169.2538   
           29                  55.4886                169.2146   
           30                  55.4250                169.1654   
           31                  55.3573                169.1062   

                Precipitable Water_q01  Precipitable Water_q99  \
Year Month Day                                                   
2009 1     1                   0.47641                 0.98104   
           2                   1.20555                 2.61017   
           3                   2.58305                 3.33054   
           4                   2.79982                 3.27254   
           5                   2.79482                 3.40021   
...                                ...                     ...   
2018 12    27                  1.64100                 4.30000   
           28                  4.00000                 4.55900  

In [9]:
tmp_quantiles.reset_index(inplace=True)
tmp_quantiles

,Year,Month,Day,Dew Point_q01,Dew Point_q99,Temperature_q01,Temperature_q99,Pressure_q01,Pressure_q99,Relative Humidity_q01,Relative Humidity_q99,Solar Zenith Angle_q01,Solar Zenith Angle_q99,Precipitable Water_q01,Precipitable Water_q99,Wind Direction_q01,Wind Direction_q99,Wind Speed_q01,Wind Speed_q99
0,2009,1,1,-4.000,1.590,-1.000,9.000,1010.0,1010.0,43.6597,92.2459,55.2478,169.0165,0.47641,0.98104,12.400,355.782,1.441,4.800
1,2009,1,2,0.410,15.000,3.000,18.000,1000.0,1010.0,81.4793,100.0000,55.1601,168.9414,1.20555,2.61017,92.988,223.218,1.100,3.900
2,2009,1,3,11.000,17.000,12.000,20.000,1000.0,1000.0,89.5569,100.0000,55.0624,168.8604,2.58305,3.33054,181.732,225.754,1.200,3.100
3,2009,1,4,11.820,16.590,12.410,21.000,1000.0,1010.0,75.9181,100.0000,54.9606,168.7694,2.79982,3.27254,181.178,252.985,0.841,3.100
4,2009,1,5,11.000,17.590,12.000,22.000,1000.0,1010.0,77.5623,100.0000,54.8529,168.6784,2.79482,3.40021,210.832,251.595,1.000,3.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2018,12,27,6.700,16.400,8.141,16.859,1008.0,1016.0,89.7891,99.2259,55.5940,169.2871,1.64100,4.30000,74.410,115.490,2.900,4.818
3648,2018,12,28,17.182,21.400,17.341,21.959,1005.0,1008.0,95.0724,100.0000,55.5404,169.2538,4.00000,4.55900,135.410,199.590,1.841,4.559
3649,2018,12,29,15.382,18.859,15.582,18.859,1008.0,1013.0,88.2540,100.0000,55.4886,169.2146,3.60000,4.40000,20.870,348.770,0.800,2.000
3650,2018,12,30,15.300,20.859,15.300,22.700,1008.0,1011.0,88.7000,100.0000,55.4250,169.1654,3.70000,4.35900,79.460,237.590,0.700,3.059


In [10]:
from tqdm import tqdm
for col in tqdm(['Wind Speed','Wind Direction','Temperature','Dew Point','Relative Humidity','Solar Zenith Angle','Precipitable Water','Pressure']):
    for yr,mth,day,low_qn,high_qn in tmp_quantiles[['Year','Month','Day',col+'_q01',col+'_q99']].values:
        tmp_df = train.query(f'Year=={yr} & Month=={mth} & Day=={day}')
        tmp_df[col] = np.clip(tmp_df[col],low_qn,high_qn)
        train.loc[tmp_df.index,col] = tmp_df[col]

  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 8/8 [04:51<00:00, 36.41s/it]


In [11]:
train.describe()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
count,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000,175295.000000
mean,2013.500265,6.523552,15.727750,11.500009,15.000086,53.065729,330.067178,254.366941,2.450880,13.559226,18.266521,1003.186177,79.383165,89.729603,2.903369,188.889664,2.263266,0.740438
std,2.871914,3.448553,8.799326,6.922225,15.000043,64.201465,370.031036,327.705377,3.080409,8.195040,8.940232,5.932098,19.654625,39.994248,1.546593,98.727256,1.278008,6.575674
min,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-17.000000,-8.000000,970.000000,7.300700,10.601300,0.123000,0.000000,0.000000,0.000000
25%,2011.000000,4.000000,8.000000,5.500000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,12.000000,1000.000000,65.620000,56.450000,1.534000,97.500000,1.300000,0.000000
50%,2013.000000,7.000000,16.000000,12.000000,30.000000,0.000000,0.000000,0.000000,0.000000,15.000000,20.000000,1000.000000,84.220000,89.860000,2.866000,200.000000,2.000000,0.000000
75%,2016.000000,10.000000,23.000000,17.500000,30.000000,103.000000,736.000000,535.000000,6.000000,21.000000,25.000000,1010.000000,97.910000,123.060000,4.222000,271.944000,3.000000,0.000000
max,2018.000000,12.000000,31.000000,23.000000,30.000000,565.000000,1040.000000,1063.000000,9.000000,26.900000,40.000000,1024.000000,100.000000,169.363100,7.600000,359.900000,12.759000,100.000000


In [12]:
train.loc[train[train['Clearsky DHI']>400].index,'Clearsky DHI'] = train.query('Year==2017 & Month==9 & Day==11')['Clearsky DHI'].quantile(0.85)

In [13]:

train.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0.0,0,0,0,0.0,5.0,1010.0,75.34,106.15,0.49900,346.1,3.1,0
1,2009,1,1,0,30,0.0,0,0,0,1.0,5.0,1010.0,80.81,112.28,0.49000,346.1,3.1,0
2,2009,1,1,1,0,0.0,0,0,4,0.0,5.0,1010.0,78.27,118.50,0.48200,347.9,3.2,0
3,2009,1,1,1,30,0.0,0,0,4,0.0,4.0,1010.0,78.27,124.78,0.47800,347.9,3.1,0
4,2009,1,1,2,0,0.0,0,0,4,0.0,4.0,1010.0,76.45,131.12,0.47641,350.0,3.0,0


In [14]:
# test = test.merge(train.groupby(['Month','Day','Hour','Minute']).agg({
#     'Max_target_Clearsky DHI':'mean', 'Min_target_Clearsky DHI':'mean',
#        'Max_target_Clearsky DNI':'mean', 'Min_target_Clearsky DNI':'mean',
#        'Max_target_Clearsky GHI':'mean', 'Min_target_Clearsky GHI':'mean'
# }).reset_index(),on=['Month','Day','Hour','Minute'],how='left')

In [15]:
df = train.append(test,ignore_index=True)
df.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0.0,0.0,0.0,0,0.0,5.0,1010.0,75.34,106.15,0.49900,346.1,3.1,0
1,2009,1,1,0,30,0.0,0.0,0.0,0,1.0,5.0,1010.0,80.81,112.28,0.49000,346.1,3.1,0
2,2009,1,1,1,0,0.0,0.0,0.0,4,0.0,5.0,1010.0,78.27,118.50,0.48200,347.9,3.2,0
3,2009,1,1,1,30,0.0,0.0,0.0,4,0.0,4.0,1010.0,78.27,124.78,0.47800,347.9,3.1,0
4,2009,1,1,2,0,0.0,0.0,0.0,4,0.0,4.0,1010.0,76.45,131.12,0.47641,350.0,3.0,0


In [16]:
df.sort_values(dt_cols,inplace=True)

In [17]:
df1 = df.groupby(['Year','Month','Day']).agg({
    'Dew Point' : ['min','max','mean'],
    'Pressure' : ['min','max'],
    'Relative Humidity' : ['min','max','mean'],
    'Precipitable Water' : ['min','max','mean'],
    'Wind Direction' : ['min','max','mean'],
    'Wind Speed' : ['min','max','mean'],
    'Fill Flag':['count']
})

df1.columns = ['YMD_'.join(col).strip() for col in df1.columns.values]
df1.reset_index(inplace=True)
df = df.merge(df1,on=['Year','Month','Day'],how='left')
df1.head()


df1 = df.groupby(['Cloud Type','Year','Month']).agg({
    'Dew Point' : ['min','max','mean'],
    'Pressure' : ['min','max'],
    'Relative Humidity' : ['min','max','mean'],
    'Precipitable Water' : ['min','max','mean'],
    'Wind Direction' : ['min','max','mean'],
    'Wind Speed' : ['min','max','mean'],
    'Fill Flag':['count']
})

df1.columns = ['CYM_'.join(col).strip() for col in df1.columns.values]
df1.reset_index(inplace=True)
df = df.merge(df1,on=['Cloud Type','Year','Month'],how='left')
df1.head()

del df1

import gc
gc.collect()

575

In [18]:
# df.sort_values(dt_cols)['Wind Direction'].diff(1)
# df.groupby(['Year']).mean()

In [19]:


!pip install pvlib



import pvlib
from pvlib import clearsky, atmosphere, solarposition
from pvlib.location import Location
from pvlib.iotools import read_tmy3
from scipy.signal import argrelextrema

     |████████████████████████████████| 27.5 MB 16.9 MB/s            


In [20]:
df.sort_values(dt_cols)['Temperature'].shift(1).rolling(2).var()
# df.groupby(['Year'])['Temperature'].diff(1)

0           NaN
1           NaN
2         0.000
3         0.000
4         0.500
          ...  
192810    0.245
192811    1.445
192812    1.620
192813    0.320
192814    0.245
Name: Temperature, Length: 192815, dtype: float64

In [21]:
dt_cols=['Year', 'Month', 'Day', 'Hour', 'Minute']

In [22]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans


def get_clearsky_feats(df):
    tus = Location(20.5937, 78.9629, 'Asia/Calcutta', 621)
    df['altitude'] = atmosphere.pres2alt(df['Pressure']*100)
    temp = pvlib.solarposition.get_solarposition(time=pd.DatetimeIndex(df['date'].values), latitude =20.5937, longitude = 78.9629, altitude=df['altitude'] , 
                                      pressure=df['Pressure'].values*100, method='nrel_numpy', temperature=df['Temperature'].values)
    
    df['zenith'] = temp['zenith'].values
    df['elevation'] = temp['elevation'].values
    df['azimuth'] = temp['azimuth'].values
    df['equation_of_time'] = temp['equation_of_time'].values
    
    
    apparent_zenith = df['Solar Zenith Angle'].values
    df['Airmass'] = pvlib.atmosphere.get_relative_airmass(df['Solar Zenith Angle'].values)
    df['Airmass'].fillna(0,inplace=True)
    # pressure = pvlib.atmosphere.alt2pres(altitude)
    df['Airmass_2'] = pvlib.atmosphere.get_absolute_airmass(df['Airmass'].values, df['Pressure'].values*100)
    df['Airmass_2'].fillna(0,inplace=True)
    df['am_h2o'] = df['Airmass_2']*df['Precipitable Water']
    
    df['Dew Point Var'] = np.exp(0.07 * df['Dew Point'] - 0.075)
    df['turbidity'] = pvlib.clearsky.lookup_linke_turbidity(pd.DatetimeIndex(df['date'].values),  20.5937, 78.9629).values
    
    df['gueymard94'] = atmosphere.gueymard94_pw(df['Temperature'], df['Relative Humidity'])
    df['transmittance'] = ((100.0 - df['Cloud Type']) / 100.0) * 0.75
    df['DNI_extra'] = pvlib.irradiance.get_extra_radiation(pd.DatetimeIndex(df['date'].values)).values
    
    df['DNI_calc'] = df['DNI_extra']*df['transmittance']**df['Airmass_2']
    
    return df

def encode(data,col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data


def feature_engg(df):
    df['date'] = pd.to_datetime(df[dt_cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns"),format="%Y-%m-%d-%H-%M")
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofYear'] = df['date'].dt.dayofyear
    df['week'] = df['date'].dt.week


    df['quarter'] = df['date'].dt.quarter
    df['ismonthstart'] = df['date'].dt.is_month_start
    df['ismonthend'] = df['date'].dt.is_month_end
    
    df['Cos Angle Val'] = np.cos(np.radians(df['Solar Zenith Angle']))
    print('Target Lags...')
    window_shifts = [2,4,12,24,48,60,120]
    
    
    for k in ['Wind Speed','Wind Direction','Temperature','Dew Point','Relative Humidity','Solar Zenith Angle','Precipitable Water','Pressure']:
        grp1 = df.sort_values(dt_cols).groupby(['Year','Month'])
        grp2 = df.sort_values(dt_cols).groupby(['Year','Month','Day'])
        grp3 = df.sort_values(dt_cols).groupby(['Cloud Type','Year','Month'])
        df['Max_target_'+k] = grp1[k].transform(lambda x : x==x.max()).apply(lambda x: 1 if x==True else 0) +grp2[k].transform(lambda x : x==x.max()).apply(lambda x: 1 if x==True else 0) 
        +grp3[k].transform(lambda x : x==x.max()).apply(lambda x: 1 if x==True else 0)

        df['Min_target_'+k] = grp1[k].transform(lambda x : x==x.min()).apply(lambda x: 1 if x==True else 0) +grp2[k].transform(lambda x : x==x.min()).apply(lambda x: 1 if x==True else 0)
        +grp3[k].transform(lambda x : x==x.min()).apply(lambda x: 1 if x==True else 0)


    for col in ['Wind Speed','Wind Direction','Temperature','Dew Point','Relative Humidity','Solar Zenith Angle','Precipitable Water','Pressure']:
        ilocs_min = argrelextrema(df[col].values, np.less_equal, order=11*12*15)[0]
        ilocs_max = argrelextrema(df[col].values, np.greater_equal, order=11*12*15)[0]
        df[col+'_min'] = 0
        df[col+'_max'] = 0
        df.loc[ilocs_min,col+'_min'] = -1
        df.loc[ilocs_max,col+'_max'] = 1
    
    print('Features Lag...')
#     for col in ['Clearsky DHI','Clearsky DNI', 'Clearsky GHI']:
#         for window in [17520+x for x in window_shifts]:
#             df[col+'_shift_'+str(window)] = df.sort_values(dt_cols)[col].shift(window)
#             df[col+'_roll_mean_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).rolling(window=2).mean()
#             df[col+'_roll_var_'+str(window)]=df.sort_values(dt_cols)[col].shift(window).rolling(2).var()
#             df[col+'_roll_std_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).rolling(window=2).std()
#             df[col+'_exp_mean_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).expanding(2).mean()
#             df[col+'_exp_var_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).expanding(2).var()
#             df[col+'_grp_diff1_'+str(window)]  = df.sort_values(dt_cols)[col].shift(window).diff(window)


    for col in ['Wind Speed','Wind Direction','Temperature','Dew Point','Relative Humidity','Cos Angle Val','Precipitable Water']:
        
        
        for col2 in ['Wind Speed','Wind Direction','Temperature','Dew Point','Relative Humidity','Cos Angle Val','Precipitable Water']:
            if col!=col2:
                df[col+'_*_'+col2] = df[col]*df[col2]
                df[col+'_d_'+col2] = df[col]/(1+df[col2])
                
                
                
        df[col+'_cont_diff_'] = df.sort_values(['Month','Day','Hour','Minute','Year'])[col].diff()
        df[col+'_cont_diff2_'] = df.sort_values(['Day','Hour','Minute','Year','Month'])[col].diff()
        print('For feature ',col)
        
        for window in window_shifts:
            df[col+'_shift_'+str(window)] = df.sort_values(dt_cols)[col].shift(window)
            df[col+'_roll_mean_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).rolling(window=window).mean()
            df[col+'_roll_var_'+str(window)]=df.sort_values(dt_cols)[col].shift(window).rolling(window).var()
            df[col+'_roll_std_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).rolling(window=window).std()
            df[col+'_exp_mean_'+str(window)] = df.sort_values(dt_cols)[col].expanding(window).mean()
            df[col+'_exp_var_'+str(window)] = df.sort_values(dt_cols)[col].expanding(window).var()
            df[col+'_grp_diff1_'+str(window)]  = df.sort_values(dt_cols)[col].diff(window)
            
#             grp_yr = df.sort_values(dt_cols).groupby(['Month'])
#             df[col+'_shift2_'+str(window)] = grp_yr[col].shift(window)
#             df[col+'_roll_mean2_'+str(window)] = grp_yr[col].shift(window).rolling(window=window).mean()
#             df[col+'_roll_var2_'+str(window)]= grp_yr[col].shift(window).rolling(window=window).var()
#             df[col+'_roll_std2_'+str(window)] = grp_yr[col].shift(window).rolling(window=window).std()
#             df[col+'_exp_mean2_'+str(window)] =grp_yr[col].shift(window).expanding(window).mean()
#             df[col+'_exp_var2_'+str(window)] =grp_yr[col].shift(window).expanding(window).var()
#             df[col+'_grp_diff12_'+str(window)]  = grp_yr[col].shift(window).diff(window)
            
            
            
            
#             for rl_wind in [2,24,120]:
                
#                 df[col+'_roll_mean2_'+str(window)] = df.sort_values(dt_cols)[col].shift(window).rolling(window=rl_wind).mean()
    df = get_clearsky_feats(df)
    
            
    print('Encoding...')    
    df = encode(df,'Month',12)
    df = encode(df,'Day',365)
    df = encode(df,'Hour',24)
    df = encode(df,'quarter',4)
    df = encode(df,'week',52)
     
    for k in dt_cols+['Cloud Type','Fill Flag']:
        df[k] = df[k].astype('category')
    

    
    df = pd.get_dummies(df,columns=['Cloud Type'],drop_first=True)
    df.drop(['date','Pressure','Fill Flag'],axis=1,inplace=True)
        
    return df

In [23]:
# for k in dt_cols+['Cloud Type']:
#     df[k] = df[k].astype('category')
df.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,...,Precipitable WaterCYM_min,Precipitable WaterCYM_max,Precipitable WaterCYM_mean,Wind DirectionCYM_min,Wind DirectionCYM_max,Wind DirectionCYM_mean,Wind SpeedCYM_min,Wind SpeedCYM_max,Wind SpeedCYM_mean,Fill FlagCYM_count
0,2009,1,1,0,0,0.0,0.0,0.0,0,0.0,...,0.26082,3.78359,1.256712,7.906,359.300,227.802052,0.3,8.659,2.399687,853
1,2009,1,1,0,30,0.0,0.0,0.0,0,1.0,...,0.26082,3.78359,1.256712,7.906,359.300,227.802052,0.3,8.659,2.399687,853
2,2009,1,1,1,0,0.0,0.0,0.0,4,0.0,...,0.44100,3.87098,1.930362,8.762,356.076,236.555257,0.4,7.500,2.871857,245
3,2009,1,1,1,30,0.0,0.0,0.0,4,0.0,...,0.44100,3.87098,1.930362,8.762,356.076,236.555257,0.4,7.500,2.871857,245
4,2009,1,1,2,0,0.0,0.0,0.0,4,0.0,...,0.44100,3.87098,1.930362,8.762,356.076,236.555257,0.4,7.500,2.871857,245


In [24]:
%%time
df = feature_engg(df)

df.head()

Target Lags...
Features Lag...
For feature  Wind Speed


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame

For feature  Wind Direction


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


For feature  Temperature
For feature  Dew Point
For feature  Relative Humidity
For feature  Cos Angle Val
For feature  Precipitable Water


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

Encoding...
CPU times: user 3min 10s, sys: 1min 36s, total: 4min 47s
Wall time: 4min 47s


,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,...,quarter_cos,week_sin,week_cos,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9
0,2009,1,1,0,0,0.0,0.0,0.0,0.0,5.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,0,0,0
1,2009,1,1,0,30,0.0,0.0,0.0,1.0,5.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,0,0,0
2,2009,1,1,1,0,0.0,0.0,0.0,0.0,5.0,...,6.123234e-17,0.120537,0.992709,0,0,1,0,0,0,0
3,2009,1,1,1,30,0.0,0.0,0.0,0.0,4.0,...,6.123234e-17,0.120537,0.992709,0,0,1,0,0,0,0
4,2009,1,1,2,0,0.0,0.0,0.0,0.0,4.0,...,6.123234e-17,0.120537,0.992709,0,0,1,0,0,0,0


In [25]:
dftrain=df[df['Clearsky DHI'].isnull()!=True]
dftest=df[df['Clearsky DHI'].isnull()==True]
dftest.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,...,quarter_cos,week_sin,week_cos,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9
175295,2019,1,1,0,0,NaN,NaN,NaN,18.4,18.8,...,6.123234e-17,0.120537,0.992709,0,0,0,0,1,0,0
175296,2019,1,1,0,30,NaN,NaN,NaN,18.4,18.6,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175297,2019,1,1,1,0,NaN,NaN,NaN,18.2,18.5,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175298,2019,1,1,1,30,NaN,NaN,NaN,18.2,18.3,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175299,2019,1,1,2,0,NaN,NaN,NaN,18.0,18.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,0,0,0


In [26]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


from collections import Counter, defaultdict

import numpy as np

from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits
from sklearn.utils.validation import check_random_state


class StratifiedGroupKFold(_BaseKFold):
    """Stratified K-Folds iterator variant with non-overlapping groups.

    This cross-validation object is a variation of StratifiedKFold that returns
    stratified folds with non-overlapping groups. The folds are made by
    preserving the percentage of samples for each class.

    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).

    The difference between GroupKFold and StratifiedGroupKFold is that
    the former attempts to create balanced folds such that the number of
    distinct groups is approximately the same in each fold, whereas
    StratifiedGroupKFold attempts to create folds which preserve the
    percentage of samples for each class.

    Read more in the :ref:`User Guide <cross_validation>`.

    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.

    shuffle : bool, default=False
        Whether to shuffle each class's samples before splitting into batches.
        Note that the samples within each split will not be shuffled.

    random_state : int or RandomState instance, default=None
        When `shuffle` is True, `random_state` affects the ordering of the
        indices, which controls the randomness of each fold for each class.
        Otherwise, leave `random_state` as `None`.
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.

    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import StratifiedGroupKFold
    >>> X = np.ones((17, 2))
    >>> y = np.array([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    >>> groups = np.array([1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 7, 8, 8])
    >>> cv = StratifiedGroupKFold(n_splits=3)
    >>> for train_idxs, test_idxs in cv.split(X, y, groups):
    ...     print("TRAIN:", groups[train_idxs])
    ...     print("      ", y[train_idxs])
    ...     print(" TEST:", groups[test_idxs])
    ...     print("      ", y[test_idxs])
    TRAIN: [2 2 4 5 5 5 5 6 6 7]
           [1 1 1 0 0 0 0 0 0 0]
     TEST: [1 1 3 3 3 8 8]
           [0 0 1 1 1 0 0]
    TRAIN: [1 1 3 3 3 4 5 5 5 5 8 8]
           [0 0 1 1 1 1 0 0 0 0 0 0]
     TEST: [2 2 6 6 7]
           [1 1 0 0 0]
    TRAIN: [1 1 2 2 3 3 3 6 6 7 8 8]
           [0 0 1 1 1 1 1 0 0 0 0 0]
     TEST: [4 5 5 5 5]
           [1 0 0 0 0]

    See also
    --------
    StratifiedKFold: Takes class information into account to build folds which
        retain class distributions (for binary or multiclass classification
        tasks).

    GroupKFold: K-fold iterator variant with non-overlapping groups.
    """

    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle,
                         random_state=random_state)

    # Implementation based on this kaggle kernel:
    # https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
    def _iter_test_indices(self, X, y, groups):
        labels_num = np.max(y) + 1
        y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
        y_distr = Counter()
        for label, group in zip(y, groups):
            y_counts_per_group[group][label] += 1
            y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        groups_and_y_counts = list(y_counts_per_group.items())
        rng = check_random_state(self.random_state)
        if self.shuffle:
            rng.shuffle(groups_and_y_counts)

        for group, y_counts in sorted(groups_and_y_counts,
                                      key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(self.n_splits):
                y_counts_per_fold[i] += y_counts
                std_per_label = []
                for label in range(labels_num):
                    std_per_label.append(np.std(
                        [y_counts_per_fold[j][label] / y_distr[label]
                         for j in range(self.n_splits)]))
                y_counts_per_fold[i] -= y_counts
                fold_eval = np.mean(std_per_label)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(group)

        for i in range(self.n_splits):
            test_indices = [idx for idx, group in enumerate(groups)
                            if group in groups_per_fold[i]]
            yield test_indices
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 rolling = False,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, rolling ,shuffle=False, random_state=None)
        self.max_train_size = max_train_size
        self.rolling = rolling

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

            
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        
        rolling_time = 0
        if self.rolling == True:
            rolling_time = group_test_starts[0]-(n_groups - n_splits * group_test_size)
            

        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            rolling_time = 0
            if self.rolling == True:
                rolling_time = group_test_start-(n_groups - n_splits * group_test_size)
            for train_group_idx in unique_groups[rolling_time : group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [27]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,GroupShuffleSplit


from catboost import CatBoostClassifier,Pool, cv
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,f1_score,mean_squared_error
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt



# targets = [ 'Clearsky DHI','Clearsky DNI', 'Clearsky GHI']
# ovr_err=[]
# for target in targets:
#     print('for Target:',target)
#     err1=[]
#     y_pred_tot1=[]
#     tscv = TimeSeriesSplit(n_splits=10)
#     # sample
#     X,y=dftrain.drop(targets,axis=1),dftrain[target]
#     Xtest =dftest.drop(targets,axis=1)
#     for train_index, test_index in tscv.split(X):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y[train_index], y[test_index]
        
#         m=LGBMRegressor(n_estimators=5000,random_state=1994,learning_rate=0.05,colsample_bytree=0.3,reg_alpha=5)
#         m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],eval_metric='MSE', early_stopping_rounds=50,verbose=50)

#         preds=m.predict(X_test,num_iteration=m.best_iteration_)
#         print("err: ",mean_squared_error(y_test,preds))
#         err1.append(mean_squared_error(y_test,preds))
#         p = m.predict(Xtest,num_iteration=m.best_iteration_)
#         y_pred_tot1.append(p)
#     print(target,' : Err : ',np.mean(err1))
#     ovr_err.append(np.mean(err1))
#     dftest[target] = np.mean(y_pred_tot1,0)
# print('Overall Err : ',np.mean(ovr_err))

In [28]:
dftrain.isnull().sum().sort_values(ascending=False)

Temperature_d_Dew Point            449
Cos Angle Val_roll_std_120         239
Precipitable Water_roll_std_120    239
Wind Speed_roll_mean_120           239
Temperature_roll_std_120           239
                                  ... 
Wind Speed_d_Precipitable Water      0
Wind Speed_*_Precipitable Water      0
Wind Speed_d_Cos Angle Val           0
Wind Speed_*_Cos Angle Val           0
Cloud Type_9                         0
Length: 562, dtype: int64

In [29]:
dftest.isnull().sum().sort_values(ascending=False)

Clearsky DHI                      17520
Clearsky DNI                      17520
Clearsky GHI                      17520
Year                                  0
Relative Humidity_roll_mean_24        0
                                  ...  
Wind Direction_roll_var_12            0
Wind Direction_roll_mean_12           0
Wind Direction_shift_12               0
Wind Direction_grp_diff1_4            0
Cloud Type_9                          0
Length: 562, dtype: int64

In [30]:
dftrain[[x for x in dftrain.columns if x.find('Clearsky DNI')!=-1]].isnull().sum()/dftrain.shape[0]

Clearsky DNI    0.0
dtype: float64

In [31]:
dftest[[x for x in dftrain.columns if x.find('Clearsky DNI')!=-1]].isnull().sum()/dftest.shape[0]

Clearsky DNI    1.0
dtype: float64

In [32]:

more_col_to_drop=['Year']
targets = [ 'Clearsky DHI','Clearsky DNI', 'Clearsky GHI']
ovr_err=[]

Xtest =dftest.drop(targets+more_col_to_drop,axis=1)                                                                                                                                                                                                                                                                                         
# Xtest = feature_engg(Xtest)
Xtest

,Month,Day,Hour,Minute,Dew Point,Temperature,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,...,quarter_cos,week_sin,week_cos,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9
175295,1,1,0,0,18.4,18.8,97.70,106.23,3.5,190.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,1,0,0
175296,1,1,0,30,18.4,18.6,98.92,112.36,3.5,187.0,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175297,1,1,1,0,18.2,18.5,98.35,118.58,3.5,184.0,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175298,1,1,1,30,18.2,18.3,99.58,124.86,3.5,185.0,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175299,1,1,2,0,18.0,18.0,99.71,131.20,3.6,186.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192810,12,31,21,30,5.6,11.1,68.64,77.89,0.8,276.0,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192811,12,31,22,0,5.7,9.3,78.31,83.06,0.8,274.0,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192812,12,31,22,30,5.7,8.5,82.64,88.34,0.8,273.0,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192813,12,31,23,0,4.8,7.8,81.49,94.37,0.8,272.0,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0


In [33]:
dftrain.isnull().sum().sort_values(ascending=False)

Temperature_d_Dew Point            449
Cos Angle Val_roll_std_120         239
Precipitable Water_roll_std_120    239
Wind Speed_roll_mean_120           239
Temperature_roll_std_120           239
                                  ... 
Wind Speed_d_Precipitable Water      0
Wind Speed_*_Precipitable Water      0
Wind Speed_d_Cos Angle Val           0
Wind Speed_*_Cos Angle Val           0
Cloud Type_9                         0
Length: 562, dtype: int64

In [34]:
dftrain[dftrain.isnull().sum()[dftrain.isnull().sum()>1].index]

,Wind Speed_shift_2,Wind Speed_roll_mean_2,Wind Speed_roll_var_2,Wind Speed_roll_std_2,Wind Speed_grp_diff1_2,Wind Speed_shift_4,Wind Speed_roll_mean_4,Wind Speed_roll_var_4,Wind Speed_roll_std_4,Wind Speed_exp_mean_4,...,Precipitable Water_exp_mean_60,Precipitable Water_exp_var_60,Precipitable Water_grp_diff1_60,Precipitable Water_shift_120,Precipitable Water_roll_mean_120,Precipitable Water_roll_var_120,Precipitable Water_roll_std_120,Precipitable Water_exp_mean_120,Precipitable Water_exp_var_120,Precipitable Water_grp_diff1_120
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.1,NaN,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.1,3.10,0.000,0.000000,0.0,NaN,NaN,NaN,NaN,3.125000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.2,3.15,0.005,0.070711,-0.2,3.1,NaN,NaN,NaN,3.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175290,4.3,4.55,0.125,0.353553,-0.8,5.0,5.150,0.010000,0.100000,2.263261,...,2.903349,2.391987,-0.2,3.9,3.548333,0.821129,0.906162,2.903349,2.391987,-0.2
175291,3.9,4.10,0.080,0.282843,-0.7,4.8,5.050,0.036667,0.191485,2.263266,...,2.903354,2.391978,0.0,3.8,3.568333,0.782474,0.884576,2.903354,2.391978,0.0
175292,3.5,3.70,0.080,0.282843,-0.9,4.3,4.825,0.149167,0.386221,2.263268,...,2.903360,2.391969,0.0,3.8,3.588333,0.743012,0.861981,2.903360,2.391969,0.0
175293,3.2,3.35,0.045,0.212132,-1.1,3.9,4.500,0.246667,0.496655,2.263267,...,2.903364,2.391959,0.0,3.8,3.608333,0.702743,0.838298,2.903364,2.391959,-0.1


In [35]:
dftest.isnull().sum()[dftest.isnull().sum()>1]

Clearsky DHI    17520
Clearsky DNI    17520
Clearsky GHI    17520
dtype: int64

In [36]:
# %%time
# warnings.filterwarnings('ignore', category=UserWarning)
# warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# # warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
# pd.set_option('mode.chained_assignment', None)
# from sklearn.preprocessing import RobustScaler
# from sklearn.utils import shuffle
# # dftrain = shuffle(dftrain,random_state=2022)

# for target in targets:
#     print('for Target:',target)
#     err1=[]
#     y_pred_tot1=[]
# #     tscv = KFold(n_splits=10,random_state=2022)
# #     tscv = GroupKFold(n_splits=10)
#     tscv = GroupTimeSeriesSplit(n_splits=6,rolling=True)
#     grp = dftrain['Year'].astype('str')+'-'+dftrain['Day'].astype('str') # day is good
#     # sample
#     X,y=dftrain.drop(targets+more_col_to_drop,axis=1),dftrain[target]
#     for train_index, test_index in tscv.split(X,y,grp):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y[train_index], y[test_index]
# #         print(X_train.index,X_test.index)
# #         sc = RobustScaler()
# #         X_train = sc.fit_transform(X_train)
# #         X_test = sc.transform(X_test)
# #         Xtest_temp = sc.transform(Xtest)
        
        
#         m=LGBMRegressor(n_estimators=5000,random_state=1994,learning_rate=0.05,reg_alpha=5,reg_lambda=200)
#         m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],eval_metric='MSE', early_stopping_rounds=50,verbose=200)

#         preds=m.predict(X_test,num_iteration=m.best_iteration_)
#         print("err: ",mean_squared_error(y_test,preds))
#         err1.append(mean_squared_error(y_test,preds))
#         p = m.predict(Xtest,num_iteration=m.best_iteration_)
#         y_pred_tot1.append(p)
#     print(target,' : Err : ',np.mean(err1))
#     ovr_err.append(np.mean(err1))
#     dftest[target] = np.mean(y_pred_tot1,0)
# print('Overall Err : ',np.mean(ovr_err))



In [37]:
%%time
warnings.filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
pd.set_option('mode.chained_assignment', None)
from sklearn.preprocessing import RobustScaler
from sklearn.utils import shuffle
# dftrain = shuffle(dftrain,random_state=2022)

for target in targets:
    print('for Target:',target)
    err1=[]
    y_pred_tot1=[]
#     tscv = KFold(n_splits=10,random_state=2022)
#     tscv = GroupKFold(n_splits=10)
    tscv = GroupTimeSeriesSplit(n_splits=15,rolling=True)
    grp = dftrain['Day'] # day is good
    # sample
    X,y=dftrain.drop(targets+more_col_to_drop,axis=1),dftrain[target]
    for train_index, test_index in tscv.split(X,y,grp):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         print(X_train.index,X_test.index)
#         sc = RobustScaler()
#         X_train = sc.fit_transform(X_train)
#         X_test = sc.transform(X_test)
#         Xtest_temp = sc.transform(Xtest)
        
        
        m=LGBMRegressor(n_estimators=5000,random_state=1994,learning_rate=0.05,reg_alpha=5,reg_lambda=200)
        m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],eval_metric='MSE', early_stopping_rounds=50,verbose=200)

        preds=m.predict(X_test,num_iteration=m.best_iteration_)
        print("err: ",mean_squared_error(y_test,preds))
        err1.append(mean_squared_error(y_test,preds))
        p = m.predict(Xtest,num_iteration=m.best_iteration_)
        y_pred_tot1.append(p)
    print(target,' : Err : ',np.mean(err1))
    ovr_err.append(np.mean(err1))
    dftest[target] = np.mean(y_pred_tot1,0)
print('Overall Err : ',np.mean(ovr_err))



for Target: Clearsky DHI
[200]	training's l2: 80.8709	valid_1's l2: 208.216
err:  207.7882339234902
[200]	training's l2: 78.8562	valid_1's l2: 206.158
err:  205.7262349554429
[200]	training's l2: 79.2365	valid_1's l2: 182.306
err:  179.99959480574879
[200]	training's l2: 77.3297	valid_1's l2: 206.057
[400]	training's l2: 45.9271	valid_1's l2: 196.46
[600]	training's l2: 32.3324	valid_1's l2: 194.41
err:  194.2252253360639
[200]	training's l2: 77.4891	valid_1's l2: 143.204
err:  142.75364395491113
[200]	training's l2: 77.1393	valid_1's l2: 158.935
[400]	training's l2: 46.0217	valid_1's l2: 155.47
err:  155.23322991403325
err:  206.49727024139364
[200]	training's l2: 76.4484	valid_1's l2: 180.237
[400]	training's l2: 45.8763	valid_1's l2: 175.627
err:  175.07754310564155
[200]	training's l2: 76.1599	valid_1's l2: 198.664
err:  197.73316158418754
[200]	training's l2: 76.9366	valid_1's l2: 157.873
[400]	training's l2: 46.4827	valid_1's l2: 153.225
[600]	training's l2: 33.1958	valid_1's l2:

In [38]:
# for Target: Clearsky DHI
# [200]	training's l2: 53.0062	valid_1's l2: 266.533
# [400]	training's l2: 24.7911	valid_1's l2: 262.536
# err:  262.35584943668135
# [200]	training's l2: 54.4576	valid_1's l2: 225.035
# [400]	training's l2: 25.7812	valid_1's l2: 220.079
# [600]	training's l2: 16.4074	valid_1's l2: 217.226
# [800]	training's l2: 11.723	valid_1's l2: 216.827
# err:  216.78909921527242
# [200]	training's l2: 54.3168	valid_1's l2: 212.006
# err:  211.7653355725129
# [200]	training's l2: 49.3922	valid_1's l2: 191.043
# err:  190.171218106937
# [200]	training's l2: 45.9837	valid_1's l2: 216.5
# err:  215.92626819281406
# [200]	training's l2: 46.2083	valid_1's l2: 237.998
# [400]	training's l2: 22.4044	valid_1's l2: 234.697
# err:  234.66753906603412
# Clearsky DHI  : Err :  221.94588493170863
# for Target: Clearsky DNI
# [200]	training's l2: 428.823	valid_1's l2: 1527.31
# [400]	training's l2: 233.914	valid_1's l2: 1505.56
# [600]	training's l2: 156.617	valid_1's l2: 1499.91
# err:  1499.0966443104157
# [200]	training's l2: 438.883	valid_1's l2: 1361.68
# [400]	training's l2: 242.285	valid_1's l2: 1335.31
# [600]	training's l2: 163.655	valid_1's l2: 1321.63
# [800]	training's l2: 119.587	valid_1's l2: 1316.66
# [1000]	training's l2: 93.2162	valid_1's l2: 1309.22
# [1200]	training's l2: 74.8333	valid_1's l2: 1306.69
# err:  1306.1068693717896
# [200]	training's l2: 438.647	valid_1's l2: 1194.48
# [400]	training's l2: 241.432	valid_1's l2: 1171.95
# err:  1171.4133161161042
# [200]	training's l2: 404.011	valid_1's l2: 1225.22
# [400]	training's l2: 220.381	valid_1's l2: 1195.57
# err:  1188.8949267514552
# [200]	training's l2: 383.439	valid_1's l2: 1131.26
# err:  1128.8252588559712
# [200]	training's l2: 378.603	valid_1's l2: 1314
# [400]	training's l2: 207.448	valid_1's l2: 1303.91
# [600]	training's l2: 140.409	valid_1's l2: 1294.02
# err:  1293.1209422031302
# Clearsky DNI  : Err :  1264.5763262681442
# for Target: Clearsky GHI
# [200]	training's l2: 25.1916	valid_1's l2: 59.491
# [400]	training's l2: 12.5901	valid_1's l2: 54.4535
# [600]	training's l2: 8.52456	valid_1's l2: 52.5712
# [800]	training's l2: 6.44253	valid_1's l2: 51.8037
# [1000]	training's l2: 5.14745	valid_1's l2: 51.2855
# [1200]	training's l2: 4.24188	valid_1's l2: 50.8816
# [1400]	training's l2: 3.57241	valid_1's l2: 50.5942
# [1600]	training's l2: 3.05723	valid_1's l2: 50.368
# [1800]	training's l2: 2.65642	valid_1's l2: 50.1952
# [2000]	training's l2: 2.30916	valid_1's l2: 49.9273
# [2200]	training's l2: 2.04703	valid_1's l2: 49.7824
# [2400]	training's l2: 1.82786	valid_1's l2: 49.6946
# [2600]	training's l2: 1.63019	valid_1's l2: 49.6197
# err:  49.6132686780055
# [200]	training's l2: 26.142	valid_1's l2: 64.1581
# [400]	training's l2: 13.1031	valid_1's l2: 56.6518
# [600]	training's l2: 8.80525	valid_1's l2: 54.3738
# [800]	training's l2: 6.62046	valid_1's l2: 53.3055
# [1000]	training's l2: 5.29092	valid_1's l2: 52.5642
# [1200]	training's l2: 4.34272	valid_1's l2: 52.1406
# [1400]	training's l2: 3.64826	valid_1's l2: 51.7228
# [1600]	training's l2: 3.12501	valid_1's l2: 51.4967
# [1800]	training's l2: 2.71402	valid_1's l2: 51.2806
# [2000]	training's l2: 2.3729	valid_1's l2: 51.0855
# [2200]	training's l2: 2.08947	valid_1's l2: 50.9189
# [2400]	training's l2: 1.86688	valid_1's l2: 50.7745
# [2600]	training's l2: 1.67204	valid_1's l2: 50.6814
# [2800]	training's l2: 1.4951	valid_1's l2: 50.5301
# [3000]	training's l2: 1.33911	valid_1's l2: 50.4348
# [3200]	training's l2: 1.20767	valid_1's l2: 50.316
# [3400]	training's l2: 1.09693	valid_1's l2: 50.26
# [3600]	training's l2: 0.996301	valid_1's l2: 50.1603
# err:  50.11128705436319
# [200]	training's l2: 27.5756	valid_1's l2: 59.5281
# [400]	training's l2: 14.0534	valid_1's l2: 53.1157
# [600]	training's l2: 9.52576	valid_1's l2: 50.9333
# [800]	training's l2: 7.18423	valid_1's l2: 49.8217
# [1000]	training's l2: 5.75474	valid_1's l2: 49.0891
# [1200]	training's l2: 4.73309	valid_1's l2: 48.5162
# [1400]	training's l2: 3.964	valid_1's l2: 47.9137
# [1600]	training's l2: 3.35972	valid_1's l2: 47.4107
# [1800]	training's l2: 2.89467	valid_1's l2: 47.1317
# [2000]	training's l2: 2.51427	valid_1's l2: 46.9608
# [2200]	training's l2: 2.20478	valid_1's l2: 46.7888
# [2400]	training's l2: 1.93892	valid_1's l2: 46.6363
# [2600]	training's l2: 1.71818	valid_1's l2: 46.5352
# [2800]	training's l2: 1.52897	valid_1's l2: 46.4086
# [3000]	training's l2: 1.37231	valid_1's l2: 46.3303
# err:  46.31779133488637
# [200]	training's l2: 25.6595	valid_1's l2: 54.3862
# [400]	training's l2: 12.8215	valid_1's l2: 46.6184
# [600]	training's l2: 8.55754	valid_1's l2: 44.3928
# [800]	training's l2: 6.40225	valid_1's l2: 43.3845
# [1000]	training's l2: 5.10718	valid_1's l2: 42.8293
# [1200]	training's l2: 4.19464	valid_1's l2: 42.4319
# [1400]	training's l2: 3.53919	valid_1's l2: 42.1475
# [1600]	training's l2: 3.03684	valid_1's l2: 41.9009
# [1800]	training's l2: 2.6288	valid_1's l2: 41.6897
# [2000]	training's l2: 2.31798	valid_1's l2: 41.5853
# [2200]	training's l2: 2.04641	valid_1's l2: 41.4217
# [2400]	training's l2: 1.82768	valid_1's l2: 41.3018
# [2600]	training's l2: 1.62523	valid_1's l2: 41.1862
# [2800]	training's l2: 1.459	valid_1's l2: 41.0772
# [3000]	training's l2: 1.30755	valid_1's l2: 40.9444
# [3200]	training's l2: 1.18004	valid_1's l2: 40.8892
# err:  40.82036124417131
# [200]	training's l2: 23.865	valid_1's l2: 55.048
# [400]	training's l2: 11.7841	valid_1's l2: 49.4541
# [600]	training's l2: 7.90595	valid_1's l2: 47.7376
# [800]	training's l2: 5.99134	valid_1's l2: 46.8621
# [1000]	training's l2: 4.79823	valid_1's l2: 46.5001
# [1200]	training's l2: 3.99248	valid_1's l2: 46.2522
# [1400]	training's l2: 3.3621	valid_1's l2: 46.0129
# [1600]	training's l2: 2.87943	valid_1's l2: 45.8048
# [1800]	training's l2: 2.47571	valid_1's l2: 45.5037
# err:  45.484741128967926
# [200]	training's l2: 24.1486	valid_1's l2: 60.0942
# [400]	training's l2: 12.1624	valid_1's l2: 53.0136
# [600]	training's l2: 8.24657	valid_1's l2: 51.4783
# [800]	training's l2: 6.10059	valid_1's l2: 50.1893
# [1000]	training's l2: 4.85258	valid_1's l2: 49.5729
# [1200]	training's l2: 3.99443	valid_1's l2: 49.125
# [1400]	training's l2: 3.32397	valid_1's l2: 48.6738
# [1600]	training's l2: 2.81042	valid_1's l2: 48.1913
# [1800]	training's l2: 2.43534	valid_1's l2: 48.0016
# [2000]	training's l2: 2.13805	valid_1's l2: 47.8607
# [2200]	training's l2: 1.88505	valid_1's l2: 47.7714
# [2400]	training's l2: 1.66557	valid_1's l2: 47.6256
# [2600]	training's l2: 1.4825	valid_1's l2: 47.5131
# [2800]	training's l2: 1.33291	valid_1's l2: 47.419
# err:  47.39499561386274
# Clearsky GHI  : Err :  46.62374084237618
# Overall Err :  511.04865068074304
# CPU times: user 2h 3min 22s, sys: 45.7 s, total: 2h 4min 8s
# Wall time: 33min 53s

In [39]:
dftest

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,...,quarter_cos,week_sin,week_cos,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9
175295,2019,1,1,0,0,0.213082,-0.891658,-0.023070,18.4,18.8,...,6.123234e-17,0.120537,0.992709,0,0,0,0,1,0,0
175296,2019,1,1,0,30,0.162404,-0.400236,-0.011240,18.4,18.6,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175297,2019,1,1,1,0,0.124666,-0.217051,0.004497,18.2,18.5,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175298,2019,1,1,1,30,0.117361,-0.158763,-0.010679,18.2,18.3,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175299,2019,1,1,2,0,0.067636,0.027373,-0.025079,18.0,18.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192810,2019,12,31,21,30,51.324585,671.518160,186.860531,5.6,11.1,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192811,2019,12,31,22,0,38.413925,502.909748,91.448960,5.7,9.3,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192812,2019,12,31,22,30,12.294087,176.733771,21.644851,5.7,8.5,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192813,2019,12,31,23,0,0.121652,6.603341,0.541237,4.8,7.8,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0


In [40]:
# dftest[targets].to_csv('mh_wipro_sub_WT0_v3.csv',index=False)
dftest


,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,...,quarter_cos,week_sin,week_cos,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9
175295,2019,1,1,0,0,0.213082,-0.891658,-0.023070,18.4,18.8,...,6.123234e-17,0.120537,0.992709,0,0,0,0,1,0,0
175296,2019,1,1,0,30,0.162404,-0.400236,-0.011240,18.4,18.6,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175297,2019,1,1,1,0,0.124666,-0.217051,0.004497,18.2,18.5,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175298,2019,1,1,1,30,0.117361,-0.158763,-0.010679,18.2,18.3,...,6.123234e-17,0.120537,0.992709,0,1,0,0,0,0,0
175299,2019,1,1,2,0,0.067636,0.027373,-0.025079,18.0,18.0,...,6.123234e-17,0.120537,0.992709,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192810,2019,12,31,21,30,51.324585,671.518160,186.860531,5.6,11.1,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192811,2019,12,31,22,0,38.413925,502.909748,91.448960,5.7,9.3,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192812,2019,12,31,22,30,12.294087,176.733771,21.644851,5.7,8.5,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0
192813,2019,12,31,23,0,0.121652,6.603341,0.541237,4.8,7.8,...,1.000000e+00,0.120537,0.992709,0,0,0,0,0,0,0


In [41]:
for k in targets:
    dftest[k] = dftest[k].apply(lambda x: 0 if x<0 else x)

In [42]:
dftest[targets].to_csv('mh_wipro_fork_tg15f_sub_new_v6.csv',index=False) #gkf

In [43]:
# mh_wipro_sub_v5.csv	26/01/2022 00:39:09	kv5	353.22801
# mh_wipro_sub_v3.csv	26/01/2022 00:19:31	kv3 with 0	362.57562
# mh_wipro_sub_WT0_v3.csv	26/01/2022 00:18:40	kv3 without 0	368.90455
# mh_wipro_sub_v2.csv	25/01/2022 23:29:14	v2 kfold	409.65866